In [1]:
import pandas as pd
import requests
import collections
import json
import ast, sys
import urllib2
import time

In [2]:
def remove_duplicate_rows(df, column_name_to_filter):
    grouped = df.groupby(column_name_to_filter)
    index = [gp_keys[0] for gp_keys in grouped.groups.values()]
    return df.reindex(index)

def convert_dict_unicode2string(data):
    if isinstance(data, basestring):
        return data.encode("utf8")
    elif isinstance(data, collections.Mapping):
        return dict(map(convert_dict_unicode2string, data.iteritems()))
    elif isinstance(data, collections.Iterable):
        return type(data)(map(convert_dict_unicode2string, data))
    else:
        return data
    
import mimetypes
import random
import string

_BOUNDARY_CHARS = string.digits + string.ascii_letters

def encode_multipart(fields, files, boundary=None):
    r"""Encode dict of form fields and dict of files as multipart/form-data.
    Return tuple of (body_string, headers_dict). Each value in files is a dict
    with required keys 'filename' and 'content', and optional 'mimetype' (if
    not specified, tries to guess mime type or uses 'application/octet-stream').

    >>> body, headers = encode_multipart({'FIELD': 'VALUE'},
    ...                                  {'FILE': {'filename': 'F.TXT', 'content': 'CONTENT'}},
    ...                                  boundary='BOUNDARY')
    >>> print('\n'.join(repr(l) for l in body.split('\r\n')))
    '--BOUNDARY'
    'Content-Disposition: form-data; name="FIELD"'
    ''
    'VALUE'
    '--BOUNDARY'
    'Content-Disposition: form-data; name="FILE"; filename="F.TXT"'
    'Content-Type: text/plain'
    ''
    'CONTENT'
    '--BOUNDARY--'
    ''
    >>> print(sorted(headers.items()))
    [('Content-Length', '193'), ('Content-Type', 'multipart/form-data; boundary=BOUNDARY')]
    >>> len(body)
    193
    """
    def escape_quote(s):
        return s.replace('"', '\\"')

    if boundary is None:
        boundary = ''.join(random.choice(_BOUNDARY_CHARS) for i in range(30))
    lines = []

    for name, value in fields.items():
        lines.extend((
            '--{0}'.format(boundary),
            'Content-Disposition: form-data; name="{0}"'.format(escape_quote(name)),
            '',
            str(value),
        ))

    for name, value in files.items():
        filename = value['filename']
        if 'mimetype' in value:
            mimetype = value['mimetype']
        else:
            mimetype = mimetypes.guess_type(filename)[0] or 'application/octet-stream'
        lines.extend((
            '--{0}'.format(boundary),
            'Content-Disposition: form-data; name="{0}"; filename="{1}"'.format(
                    escape_quote(name), escape_quote(filename)),
            'Content-Type: {0}'.format(mimetype),
            '',
            value['content'],
        ))

    lines.extend((
        '--{0}--'.format(boundary),
        '',
    ))
    body = '\r\n'.join(lines)

    headers = {
        'Content-Type': 'multipart/form-data; boundary={0}'.format(boundary),
        'Content-Length': str(len(body)),
    }

    return (body, headers)

## Formatting output from "Session Buddy"

In [23]:
with open('data.json') as f:
    string = ""
    for line in f:
        string += line
        
string = string.decode('ascii', 'ignore')
        
data = json.loads(unicode(string))
df = pd.DataFrame(data["tabs"])

df

,active,audible,autoDiscardable,discarded,favIconUrl,height,highlighted,id,incognito,index,mutedInfo,pinned,selected,status,title,url,width,windowId
0,False,False,True,False,https://muricoca.github.io/crab/static/favicon...,950,False,5043,False,9,{u'muted': False},False,False,complete,2. Getting started: an introduction to recomme...,https://muricoca.github.io/crab/tutorial.html,1920,3000
1,False,False,True,False,https://www.saagie.com/favicon.ico,950,False,5047,False,10,{u'muted': False},False,False,complete,Machine Learning for Grandmas,https://www.saagie.com/fr/blog/machine-learnin...,1920,3000
2,False,False,True,False,NaN,950,False,5083,False,11,{u'muted': False},False,False,complete,TensorFlow in a NutshellPart Two: HybridLearni...,http://camron.xyz/index.php/2016/09/13/hybrid_...,1920,3000
3,False,False,True,False,NaN,950,False,5087,False,12,{u'muted': False},False,False,complete,TensorFlow in a NutshellPart One: Basics Camr...,http://camron.xyz/index.php/2016/08/22/in_a_nu...,1920,3000
4,False,False,True,False,NaN,950,False,5095,False,13,{u'muted': False},False,False,complete,TensorFlow in a NutshellPart Three: All theMod...,http://camron.xyz/index.php/2016/10/03/nutshel...,1920,3000
5,False,False,True,False,NaN,950,False,5091,False,14,{u'muted': False},False,False,complete,Sales Automation Through a Deep Learning Platf...,http://camron.xyz/index.php/2016/09/22/sales-a...,1920,3000
6,False,False,True,False,NaN,950,False,5108,False,15,{u'muted': False},False,False,complete,Deep Learning Cheat Sheet Camron's Blog,http://camron.xyz/index.php/2016/12/06/deep-le...,1920,3000
7,False,False,True,False,https://assets-cdn.github.com/favicon.ico,950,False,5075,False,16,{u'muted': False},False,False,complete,c0cky/TensorFlow-in-a-Nutshell: The repo for t...,https://github.com/c0cky/TensorFlow-in-a-Nutshell,1920,3000


In [24]:
output_df = df[['title','url']]
output_df.insert( 2,'user', "dataradar")
output_df.insert( 3,'tags',"")
output_df.insert( 4,'twitter',True)
output_df.insert( 5,'autoformat',False)
output_df.insert( 6,'activated',True)

output_df = remove_duplicate_rows(output_df, 'url')

In [25]:
output_df.to_json("output.json", orient="records", force_ascii=False)

## Check Uniqueness in consolidated.json File

In [29]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

df = remove_duplicate_rows(df, 'url')

In [30]:
api_route = "https://www.feedcrunch.io/api/1.0/authenticated/get/article/exists/"

apikey_local = "91ce86bd-ae46-4250-b599-727c1970b609"
apikey_fc    = "4377190f-24a5-4ef7-a761-a877bf6218e3"

api_url = api_route + apikey_fc + "/"

def check_link_exists(url, user):
    r = requests.get(api_url+'?link='+url+'&posting_user='+user)
    try:
        return r.json()["exists"]
    except:
        return False

In [31]:
lines_2_delete = []

for index in range(df.shape[0]):
    
    url = df.iloc[index]['url']
    user = df.iloc[index]['user']
    
    if check_link_exists(url, user):
        lines_2_delete.append(index)
        print 'Duplicate Found: ' + url + " && user: " + user

Duplicate Found: https://tryolabs.com/blog/2016/12/06/major-advancements-deep-learning-2016/ && user: dataradar
Duplicate Found: http://camron.xyz/index.php/2016/08/22/in_a_nutshell_part_one/ && user: dataradar
Duplicate Found: https://www.datacamp.com/community/tutorials/machine-learning-python && user: dataradar
Duplicate Found: http://camron.xyz/index.php/2016/12/06/deep-learning-cheat-sheet/ && user: dataradar
Duplicate Found: https://research.googleblog.com/2016/12/open-sourcing-embedding-projector-tool.html && user: dataradar
Duplicate Found: http://www.dataminingapps.com/2016/09/a-basic-introduction-to-deep-learning/ && user: dataradar
Duplicate Found: http://camron.xyz/index.php/2016/10/03/nutshell-all-the-models/ && user: dataradar
Duplicate Found: http://www.benkuhn.net/squared && user: dataradar
Duplicate Found: http://camron.xyz/index.php/2016/09/13/hybrid_learning/ && user: dataradar
Duplicate Found: https://blog.monkeylearn.com/the-definitive-guide-to-natural-language-pro

In [32]:
df.drop(df.index[lines_2_delete], inplace=True)

df = df.sort_values(by=['url', 'title'], ascending=[True, True])

In [33]:
df.to_json("consolidated.json", orient="records", force_ascii=False)

## Describe Data

In [34]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

print "There are " + str(df.shape[0]) + " articles in dataset"
df = remove_duplicate_rows(df, 'url')
print "There are " + str(df.shape[0]) + " articles in dataset without duplicates"

There are 58 articles in dataset
There are 58 articles in dataset without duplicates


## Sorting Data

In [35]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

df = remove_duplicate_rows(df, 'url') 
df = df.sort_values(by=['url', 'title'], ascending=[True, True])
#df = df.sort_values(by=['title', 'url'], ascending=[True, True])

df.to_json("consolidated.json", orient="records", force_ascii=False)
df

,activated,autoformat,tags,title,twitter,url,user
0,True,True,,"PostgreSQL anti-patterns - read, modify, write...",True,http://blog.2ndquadrant.com/postgresql-anti-pa...,dataradar
1,True,False,,Engineering is the bottleneck in (Deep Learnin...,True,http://blog.dennybritz.com/2017/01/17/engineer...,dataradar
2,True,False,,"Santander Product Recommendation Competition, ...",True,http://blog.kaggle.com/2017/01/12/santander-pr...,dataradar
3,True,False,,Statistics for Hackers,True,http://christopherroach.com/articles/statistic...,dataradar
4,True,False,,Pythonic Linear Regression,True,http://francescopochetti.com/pythonic-linear-r...,dataradar
5,True,False,,Simple Matrix Factorization with TensorFlow,True,http://hameddaily.blogspot.fr/2016/12/simple-m...,dataradar
6,True,False,,How To Backtest Machine Learning Models for Ti...,True,http://machinelearningmastery.com/backtest-mac...,dataradar
7,True,False,,How to Use and Remove Trend Information from T...,True,http://machinelearningmastery.com/time-series-...,dataradar
8,True,True,,State of the art deep learning model for quest...,True,http://metamind.io/research/state-of-the-art-d...,dataradar
9,True,False,,Playing with 80 Million Amazon Product Review ...,True,http://minimaxir.com/2017/01/amazon-spark/,dataradar


## Hashtag Automation

In [36]:
max_hashtags = 5

def hashtags_free_number(df, index):
    
    tags = df.iloc[index]['tags']
        
    str_list = tags.split(",")
    str_list = filter(None, str_list) # fastest
    
    return max_hashtags - len(str_list)

In [37]:
def check_substr(input_str, keyword):
    
    tmp_str = str.lower(input_str)   
    tmp_keyword = str.lower(keyword)
    
    if keyword[-1:] == '-':
        
        tmp_str = str.replace(tmp_str, ' ', '')
        tmp_str = str.replace(tmp_str, '-', '')
        tmp_str = str.replace(tmp_str, '_', '')
        tmp_str = str.replace(tmp_str, '/', '')
        tmp_str = str.replace(tmp_str, '#', '')
        tmp_str = str.replace(tmp_str, '?', '')

        
        tmp_keyword = str.replace(tmp_keyword, ' ', '')
        tmp_keyword = str.replace(tmp_keyword, '-', '')
        tmp_keyword = str.replace(tmp_keyword, '_', '')
        tmp_keyword = str.replace(tmp_keyword, '/', '')
        tmp_keyword = str.replace(tmp_keyword, '#', '')
        tmp_keyword = str.replace(tmp_keyword, '?', '')
    
    
    if tmp_keyword in tmp_str:
        return True
    else:
        return False

In [38]:
def check_pattern(df, index, pattern):
    
    if check_substr(df.iloc[index]['title'].encode("utf-8"), pattern):
        return True
    else:
        return False

In [39]:
def set_keyword(df, index, keyword_list, pattern = ""):
     
    if not check_pattern(df, index, pattern):
        return False
        
    tags = df.iloc[index]['tags']
        
    str_list = tags.split(",")
    str_list = filter(None, str_list) # fastest
    
    str_list = [s.encode('ascii') for s in str_list] # unicode to string
    
    old_tmp = str_list[:]
    
    for keyword in keyword_list:
        if len(str_list) >= max_hashtags:
            break
        if keyword not in str_list:
            str_list.append(keyword)
    
    df.set_value(index, "tags", ",".join(str_list))
    return max_hashtags - len(str_list)

In [40]:
def write_stats(df, posts_full, full_at_start):
    data = []
    total_posts = df.shape[0]
    print "Posts Full = " + str(posts_full)+"/"+str(df.shape[0])
    print "Posts Full at Start = " + str(full_at_start)+"/"+str(df.shape[0])

    for i in range (100):
        data.append(0)

    for index in range(df.shape[0]):
        tags = df.iloc[index]['tags']

        str_list = tags.split(",")
        str_list = filter(None, str_list) # fastest

        data[len(str_list)] += 1

    for i in range (100):
        if data[i] != 0:
            print "Number of rows with " + str(i) + " Hashtags : " + str(data[i]) + "/" + str(total_posts)

In [44]:
##################### LOADING JSON ##################################
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

df = remove_duplicate_rows(df, 'url') 
df = df.sort_values(by=['url', 'title'], ascending=[True, True])

###################### PARSING JSON #################################
posts_full = 0
full_at_start = 0

for index in range(df.shape[0]):
    max_iteration = hashtags_free_number(df, index)
    
    if max_iteration > 0:
        for pattern, keywords_list in keywords.iteritems():  
            tmp_rslt = set_keyword(df, index, keywords_list, pattern)
            
            if tmp_rslt is not False:
                max_iteration = tmp_rslt
            
                if max_iteration <= 0:
                    break
        
        #"""
        if max_iteration > 0 and df.iloc[index]['user'] == "dataradar":
            
            Last_Keywords = ['Data', 'DataScience','AI']            
 
            tmp_rslt = set_keyword(df, index, Last_Keywords, "")

            if tmp_rslt is not False:
                max_iteration = tmp_rslt
        #"""
                        
        if max_iteration <= 0:
            posts_full += 1
        elif max_iteration == 5:
            print df.iloc[index]['title'] 
            print df.iloc[index]['tags']
            print "###########"
        
    else:
        full_at_start += 1
        posts_full += 1

write_stats(df, posts_full, full_at_start)

Posts Full = 41/58
Posts Full at Start = 0/58
Number of rows with 1 Hashtags : 1/58
Number of rows with 3 Hashtags : 7/58
Number of rows with 4 Hashtags : 9/58
Number of rows with 5 Hashtags : 41/58


In [45]:
df.to_json("consolidated.json", orient="records", force_ascii=False)
df

,activated,autoformat,tags,title,twitter,url,user
0,True,True,"Database,PostgreSQL,Data,DataScience,AI","PostgreSQL anti-patterns - read, modify, write...",True,http://blog.2ndquadrant.com/postgresql-anti-pa...,dataradar
1,True,False,"DeepLearning,MachineLearning,NeuralNet,Library...",Engineering is the bottleneck in (Deep Learnin...,True,http://blog.dennybritz.com/2017/01/17/engineer...,dataradar
2,True,False,"Recommendation,RecommendationEngine,Recommende...","Santander Product Recommendation Competition, ...",True,http://blog.kaggle.com/2017/01/12/santander-pr...,dataradar
3,True,False,"Data,DataScience,AI",Statistics for Hackers,True,http://christopherroach.com/articles/statistic...,dataradar
4,True,False,"Python,Data,DataScience,AI",Pythonic Linear Regression,True,http://francescopochetti.com/pythonic-linear-r...,dataradar
5,True,False,"Simple,TensorFlow,DeepLearning,MachineLearning...",Simple Matrix Factorization with TensorFlow,True,http://hameddaily.blogspot.fr/2016/12/simple-m...,dataradar
6,True,False,"Test,HowTo,GettingStarted,Forecasting,TimeSeries",How To Backtest Machine Learning Models for Ti...,True,http://machinelearningmastery.com/backtest-mac...,dataradar
7,True,False,"Python,HowTo,GettingStarted,Data,DataScience",How to Use and Remove Trend Information from T...,True,http://machinelearningmastery.com/time-series-...,dataradar
8,True,True,"DeepLearning,MachineLearning,NeuralNet,Library...",State of the art deep learning model for quest...,True,http://metamind.io/research/state-of-the-art-d...,dataradar
9,True,False,"Product,Amazon,Data,DataScience,AI",Playing with 80 Million Amazon Product Review ...,True,http://minimaxir.com/2017/01/amazon-spark/,dataradar


In [43]:
keywords = {
    # DataRadar Oriented Tags 
    'Accuracy': ['Accuracy'],
    'Adversa': ['GAN', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'Analysis': ['Analysis'],
    'Analytic': ['Analytic'],
    'Arxiv': ['Arxiv', 'Research'],
    'AutoEncoder': ['AE', 'AutoEncoder', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'Bayes': ['Bayes', 'MachineLearning', 'Statistic'],
    'BigData-': ['BigData'],
    'Boosting': ['Boosting', 'MachineLearning'],
    'CNN ': ['CNN', 'ConvNet', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'CUDA ': ['CUDA', 'GPU', 'NVidia'],
    'Caffe ': ['Caffe', 'DeepLearning', 'MachineLearning', 'NeuralNet', 'Library'],
    'Cassandra': ['Cassandra', 'Apache', 'BigData'],
    'Classifi': ['Classification', 'MachineLearning'],
    'Clustering': ['Clustering', 'MachineLearning'],
    'Cohort': ['Analysis', 'Cohort', 'Statistic'],
    'ComputerVision-': ['ComputerVision', 'MachineLearning'],
    'ConvNet': ['CNN', 'ConvNet', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'Convol': ['CNN', 'ConvNet', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'CrossValidation-': ['CrossValidation', 'Statistic', 'MachineLearning'],
    'CuDNN ': ['CuDNN', 'CUDA', 'GPU', 'NVidia'],
    'DNN ': ['DNN', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'DSSTNE ': ['Amazon', 'DSSTNE', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'Data': ['Data', 'DataScience'],
    'DataAnalysis-': ['DataAnalysis', 'MachineLearning', 'Statistic'],
    'DataAnalyst-': ['DataScientist', 'DataAnalyst', 'Job'],
    'DataLake-': ['DataLake', 'BigData'],
    'DataScience-': ['DataScience', 'Data'],
    'DataScientist-': ['DataScientist', 'DataAnalyst', 'Job'],
    'DataStructure-': ['DataStructure'],
    'DataViz-': ['DataViz'],
    'Dataset': ['Dataset'],
    'DeepDream-': ['DeepDream', 'Google', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'DeepLearning-': ['DeepLearning', 'MachineLearning', 'NeuralNet', 'Library'],
    'Diagram': ['diagram', 'DataViz'],
    'Distribution': ['Statistic', 'Probability', 'Distribution'],
    'ElasticSearch-': ['ElasticSearch', 'BigData'],
    'EnsembleLearning-': ['EnsembleLearning', 'MachineLearning'],
    'EnsembleModel-': ['EnsembleLearning', 'MachineLearning'],
    'FPGA ': ['FPGA'],
    'FaceDetection-': ['FaceDetection', 'ComputerVision'],
    'FaceRecognition-': ['FaceRecognition', 'ComputerVision'],
    'FeatureExtract-': ['FeatureExtraction', 'MachineLearning'],
    'FeatureSelect-': ['FeatureSelection', 'MachineLearning'],
    'Flink ': ['Flink', 'Apache', 'BigData'],
    'Forecasting': ["Forecasting"],
    'GAN ': ['GAN', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'Generali': ['Generalisation', 'MachineLearning'],
    'GenerativeModel-': ['GenerativeModel', 'MachineLearning'],
    'GettingStarted-': ['GettingStarted'],
    'Gradient': ['Gradient', 'MachineLearning'],
    'GradientBoosting-': ['GradientBoosting', 'MachineLearning'],
    'GradientDescent-': ['GradientDescent', 'MachineLearning'],
    'Guide': ['Guide'],
    'Hadoop': ['Hadoop', 'Apache', 'BigData'],
    'HowTo-': ['HowTo', 'GettingStarted'],
    'HyperParam-': ['HyperParameter', 'MachineLearning'],
    'ImageAnaly-': ['ImageAnalysis', 'ComputerVision', 'MachineLearning'],
    'ImageCaptio-': ['ImageCaptioning', 'ComputerVision', 'MachineLearning'],
    'ImageClassifi-': ['ImageClassification', 'ComputerVision', 'MachineLearning'],
    'ImageProcess-': ['ImageProcessing', 'ComputerVision', 'MachineLearning'],
    'ImageRecogni-': ['ImageRecognition', 'ComputerVision', 'MachineLearning'],
    'ImageSegment-': ['ImageSegmentation', 'ComputerVision', 'MachineLearning'],
    'ImageUnderst-': ['ImageUnderstanding', 'ComputerVision', 'MachineLearning'],
    'Kafka ': ['Kafka', 'Apache', 'Distributed', "Streaming"],
    'KMeans ': ['KMeans', 'MachineLearning', 'Clustering'],
    'KNN ': ['KNN', 'MachineLearning', 'Classification'],
    'Kaggle': ['Competition', 'Kaggle', 'MachineLearning'],
    'Keras ': ['DeepLearning', 'Keras', 'MachineLearning', 'NeuralNet', 'Library'],
    'LDA ': ['LDA', 'MachineLearning', 'FeatureSelection'],
    'LSTM ': ['DeepLearning', 'LSTM', 'MachineLearning', 'RNN', 'NeuralNet'],
    'Lasagne': ['DeepLearning', 'Lasagne', 'MachineLearning', 'NeuralNet', 'Library'],
    'MNIST ': ['MNIST'],
    'MXNet ': ['DeepLearning', 'MXNet', 'MachineLearning', 'NeuralNet', 'Library'],
    'MachineLearning-': ['MachineLearning'],
    'MariaDB': ['Database', 'MariaDB'],
    'Markov ': ['MarkovChain', 'MachineLearning', 'NeuralNet'],
    'Mathemati': ['Mathematic', 'Science'],
    'MissingVal-': ['MissingValue'],
    'Model': ['Model'],
    'MongoDB': ['Database', 'MongoDB'],
    'MonteCarlo-': ['MonteCarlo', 'Stochastic'],
    'Multilayer-': [' MultiLayer', 'DeepLearning', 'NeuralNet'],
    'MultiClass-': ['MultiClass', 'MachineLearning'],
    'MultiLabel-': ['MultiClass', 'MachineLearning'],
    'MySQL': ['Database', 'MySQL'],
    'NLP ': ['NLP', 'DeepLearning', 'MachineLearning'],
    'NVidia': ['NVidia'],
    'Neo4J': ['Neo4J', 'Database', 'GraphDB'],
    'Neural': ['NeuralNet', 'DeepLearning', 'MachineLearning'],
    'NeuralNet-': ['NeuralNet', 'DeepLearning', 'MachineLearning'],
    'NoSQL': ['NoSQL', 'Database', 'BigData'],
    'NonLinear-': ['NonLinear', 'MachineLearning'],
    'Numpy': ['Numpy', 'Python', 'Library'],
    'ObjectDetec-': ['ObjectDetection', 'ComputerVision', 'DeepLearning', 'MachineLearning'],
    'ObjectReco-': ['ObjectRecognition', 'ComputerVision', 'DeepLearning', 'MachineLearning'],
    'ObjectSegmen-': ['ObjectSegmentation', 'ComputerVision', 'DeepLearning', 'MachineLearning'],
    'Pandas': ['Pandas', 'Python', 'Library', 'DataManipulation'],
    'Pipeline': ['DataPipeline', 'MachineLearning'],
    'PostgreSQL': ['Database', 'PostgreSQL'],
    'Proba': ['Probability', 'Statistic'],
    'Propaga': ['BackPropagation', 'DeepLearning', 'NeuralNet'],
    'Python': ['Python'],
    'RSS ': ['RSS', 'Feed'],
    'Recurrent': ['DeepLearning', 'MachineLearning', 'RNN', 'NeuralNet'],
    'SVM ': ['MachineLearning', 'SVM', 'Classification'],
    'Scikit': ['Python', 'Scikit', 'MachineLearning', 'Library'],
    'Scipy': ['Python', 'Scipy', 'MachineLearning', 'Library'],
    'Segmentation': ["Segmentation"],
    'SelectFeatu-': ['FeatureSelection', 'MachineLearning'],
    'Sentiment': ['SentimentAnalysis', 'NLP', 'MachineLearning'],
    'Spark ': ['Apache', 'Spark', 'BigData'],
    'Streaming': ['Streaming', 'MachineLearning'],
    'TensorFlow': ['TensorFlow', 'DeepLearning', 'MachineLearning', 'NeuralNet', 'Library'],
    'Tensor': ['TensorFlow', 'DeepLearning', 'MachineLearning', 'NeuralNet', 'Library'],
    'TextMining-': ['TextMining', 'MachineLearning'],
    'TextProcess-': ['TextProcessing', 'MachineLearning'],
    'TimeSerie-': ["TimeSeries"],
    'Theano': ['DeepLearning', 'MachineLearning', 'Theano', 'Library'],
    'Tutorial': ['Tutorial', 'GettingStarted'],
    'Visuali': ['DataViz', 'MachineLearning'],
    'Vision': ['ComputerVision', 'MachineLearning'],
     
    # Engineering Oriented Tags 
    '2D': ["2D"],
    '3D': ['3D'],
    '3DModel-': ['3DModel', '3D'],
    'ABTest-': ['ABTest'],
    'API ': ['API'],
    'AWS ': ['AWS', 'Amazon', 'Cloud'],
    'AWSLambda-': ['AWSLambda', 'ServerLess', 'AWS', 'Amazon', 'Cloud'],
    'Academic': ['Academic', 'Research'],
    'Administration': ['Administration'],
    'AddOn-': ['AddOn'],
    'Adventure': ["Adventure"],
    'Agile': ['Agile'],
    'Algorithm': ['Algorithm', 'ComputerScience'],
    'Amazon': ['Amazon'],
    'AmazonWebService-': ['AWS', 'Amazon', 'Cloud', 'SaaS', 'PaaS'],
    'Android': ['Android', 'Mobile', 'Smartphone'],
    'Angular': ['Angular', 'JS', 'WebApp'],
    'Animation': ['Animation', 'CSS', 'WebDevelopment', 'WebApp'],
    'Approach': ['Approach'],
    'Async': ['Async'],
    'AutoScaling-': ['AutoScaling'],
    'Automation': ['Automation'],
    'Backdoor': ['Backdoor', 'Security', 'ITSec', 'InfoSec'],
    'Backup': ['Backup'],
    'Bash ': ['Bash', 'Script', 'Linux'],
    'Beautiful ':["Beautiful"],
    'BeautifulSoup-': ['BeautifulSoup', 'Python'],
    'Behavior': ['Behavior'],
    'Benchmark': ['Benchmark'],
    'Botnet ': ['Botnet', 'ITSecurity', 'InfoSec'],
    'Browser': ["Browser"],
    'Business': ['Business'],
    'Code ': ['Code', 'DevLife'],
    'C++ ': ['C++'],
    'CPU ': ['CPU'],
    'CSS ': ['CSS', 'WebApp'],
    'CSS3 ': ['CSS3'],
    'Cache ': ['Caching'],
    'Caching': ['Caching'],
    'Certificate': ['Certificate', 'Security', 'Encryption', 'ITSec', 'InfoSec'],
    'ChatBot-': ['ChatBot'],
    'Cheatsheet': ['Cheatsheet'],
    'CleanEnergy-': ["CleanEnergy", "GreenEnergy"],
    'CodeEditor-': ['CodeEditor','Code', 'DevLife'],
    'Collab': ['Collaborative'],
    'Compiler': ["Compiler"],
    'CommandLine-': ['CommandLine'],
    'Commit': ['Commit', 'Git', 'SVN'],
    'Compensation': ['Compensation', 'Gratification', 'Job', 'Salary'],
    'Computing': ['Computing', 'ComputerScience'],
    'Container': ['Container'],
    'Control': ['Control'],
    'CPython': ['CPython', 'Python', 'Performance', 'Optimization'],	 
    'Crypto': ['Cryptography', 'Security', 'Encryption'],
    'Customer': ["Customer"],
    'Dashboard': ["Dashboard"],
    'Decorator': ['Decorator', 'Code'],
    'DesignPattern-': ['DesignPattern', "Code", 'DevLife', 'BestPratice', 'SoftwareArchitecture'],
    'Design': ['Design'],
    'Developer': ['Developer', 'Job'],
    'Development': ['Development'],
    'DevelopmentCycle-': ["DevelopmentCycle"],
    'Django': ['Django', 'Python', 'WebApp', 'ORM'],
    'Docker': ['Docker', 'Container', 'Virtualisation', 'DataCenter', 'VM'],
    'Document': ['Document'],
    'Encrypt': ['Encryption', 'Security', 'ITSec', 'InfoSec'],
    'Energy': ["Energy"],
    'Excel': ['Excel', 'SpreadSheet', 'Microsoft', 'Office'],
    'Expert': ["Expert"],
    'Facebook': ['Facebook'],
    'Framework': ['Framework'],
    'Fundamental': ['Fundamental'],
    'FundRaising': ['FundRaising'],
    'Funding': ['Funding'],
    'GCE ': ['GCE', 'Google', 'Cloud'],
    'GCP ': ['GCP', 'Google', 'Cloud'],
    'GPU ': ['GPU'],
    'Galler': ['Gallery', 'Photo', 'Image'],
    'Git': ['Git', 'OpenSource'],
    'Github': ['Github', 'Git', 'OpenSource'],
    'Gitlab': ['Gitlab', 'Git', 'OpenSource'],
    'GoldenRule': ['GoldenRule'],
    'Google': ['Google'],
    'GoogleCloudPlatform-': ['GCP', 'Google', 'Cloud', 'SaaS', 'PaaS'],
    'GrowthHacking-': ['GrowthHacking'],
    'Hash ': ["Hashing", 'Hash', 'Cryptography', "ITSec", "InfoSec"],
    'HTML ': ['HTML', 'WebApp'],
    'HTML5 ': ["HTML5"],
    'HTTPS ': ['HTTPS', 'Encryption', 'Security'],
    'Handbook': ['Handbook'],
    'Haskell': ['Haskell'],
    'HumanCost-': ["HumanCost"],
    'IPython': ['IPython', 'Notebook', 'Python'],
    'Industry': ['Industry'],
    'Interactive': ['Interactive'],
    'InternetOfThi-': ['IoT'],
    'Interview': ['Interview'],
    'IoT ': ['IoT'],
    'Introduc': ['Introduction'],	 
    'JQuery': ['JQuery', 'JS', 'WebApp'],
    'JS ': ['JS', 'WebApp'],
    'JSON ': ['JSON'],
    'Java ': ['Java'],
    'Javascript': ['JS', 'WebApp'],
    'Jupyter': ['Jupyter', 'Notebook', 'Python'],
    'Kubernetes': ['Kubernetes', 'Docker','Orchestration'],
    'Language': ["Language"],
    'LaTeX': ['LaTeX'],
    'Log': ["Log"], 
    'Mac ': ['Mac', 'MacOS', 'Apple'],
    'MOOC': ['MOOC', 'Learning'],
    'MVP ': ['MVP', 'Startup'],
    'MagicLeap-': ['MagicLeap', 'VR', 'AR'],
    'MagicMirror-': ['MagicMirror'],
    'Markup': ['Markup'],
    'Memory': ['Memory', 'Performance'],
    'Messag': ['Message'],
    'Metasploit': ['Metasploit', 'Hack', 'Security', 'ITSec', 'InfoSec'],
    'Microservice-': ['MicroService', 'ServerLess', 'Cloud', 'SaaS'],
    'Microsoft': ['Microsoft'],
    'Migration': ['Migration', 'Database', 'ORM'],
    'Monitor': ['Monitoring', 'Service'],
    'Motion': ['Motion'],
    'NodeJS': ['NodeJS', 'JS', 'WebApp'],
    'Notebook': ['Notebook', 'Notebook', 'Python'],
    'Notif': ['Notification'],
    'OAuth': ['OAuth', 'Authentication'],
    'Online': ['Online'],
    'OpenCV': ['OpenCV', 'ComputerVision', 'C++', 'Python'],
    'OpenGL': ['OpenGL'],
    'OpenID': ['OpenID', 'Authentication'],
    'OpenSource-': ['OpenSource', 'Software'],
    'Opportunit': ["Opportunity"],
    'Orchestration': ["Orchestration"],
    'Personal': ['Personal'],
    'PHP ': ['PHP', 'WebApp'],
    'Pass': ['Password', 'Authentication'],
    'Performance': ['Performance'],
    'Pitch': ['Pitch', 'Startup', 'ElevatorPitch'],
    'Process': ['Process'],
    'Productiv': ['Productivity', 'Efficiency', 'Performance'],
    'Product ': ["Product"], # Don't remove the space
    'Programmer': ['Programmer', 'DevLife', 'ComputerScience'],
    'Programming': ["Programmer", 'DevLife', 'ComputerScience'],
    'Proof': ['Proof'],
    'Publish': ['Publishing'],
    'Push': ['Push'],
    'Radiobox': ["Radiobox"],
    'Reddit': ["Reddit"], 
    'REST ': ['REST'],
    'RSS ': ['RSS', 'Feed'],
    'Random': ['Random'],
    'Raspberry': ['RaspberryPi', 'CheapComputing', 'EmbeddedDevice'],
    'React': ['React', 'ReactJS', 'Javascript', 'WebApp', 'ReactNative'],
    'Recommendation': ['Recommendation', 'RecommendationEngine', 'RecommenderSystem'],
    'RegEx': ['Regex'],
    'RegularExpr-': ['Regex'],
    'RemoteAccess': ['RemoteAccess', 'RDP'],
    'Render': ['Render'],
    'Resilient': ['Resilient'],
    'SAML ': ['SAML', 'Authentication'],
    'SMS ': ['SMS', 'Message'],
    'Scala': ['Scala'],
    'Scientific': ['Scientific'],
    'Science': ['Science'],
    'Script': ['Script'],
    'Search': ['Search'],
    'SearchEngine-': ['SearchEngine'],
    'Securi': ['ITSec', 'Security', 'InfoSec'],
    'Server': ['Server'],
    'ServerLess-': ['ServerLess'],
    'Shell ': ['Shell'],
    'SideProject-': ['SideProject'],
    'Simple': ["Simple"],
    'Smart ': ['Smart'],
    'Software': ['Software'],
    'SSL ': ['SSL', 'Encryption', 'ITSec', 'InfoSec'],
    'Startup': ['Startup'],
    'Static': ['Static'],
    'Storage': ['Storage'],
    'SubString-': ['SubString'], 
    'Suggestion': ['Recommendation', 'RecommendationEngine', 'RecommenderSystem'],
    'System': ['System'],
    'Teach': ['Teach'],
    'TechDebt-': ['TechDebt'],
    'TechicalDebt-': ['TechDebt'],
    'Test': ['Test'],
    'Tethering': ['Tethering'],
    'Theory': ["Theory"],
    'Tool': ['Tool'],
    'Tracking': ['Tracking'],
    'UI ': ['UI', 'UX', 'Design'],
    'UX ': ['UI', 'UX', 'Design'],
    'VC ': ['VC', 'FundRaising', 'Investor'],
    'Visitor': ["Visitor"],
    'Web ': ['Web'], #Don't remove spaces
    'WebApp-': ['WebApp', 'WebDevelopment'],
    'WebDev-': ['WebDevelopment', 'WebApp'],
    'WebDesign': ["WebDesign", 'Design', 'UI', 'UX'],	
    'WebDesigner': ['WebDesigner', "WebDesign", 'Design', 'UI', 'UX'],
    'WebGL': ['WebGL'],
    'WebSocket-': ['WebSocket', 'Web', 'Async'],
    'Website': ['WebDevelopment', 'WebApp'],
    'Wireframe': ["Wireframe"],
}

## Rename Column "URL" to "LINK" before UPLOAD

In [6]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

df=df.rename(columns = {'url':'link'})

df.to_json("consolidated.json", orient="records", force_ascii=False)
df

,activated,autoformat,tags,title,twitter,link,user
0,True,True,"Database,PostgreSQL,Data,DataScience,AI","PostgreSQL anti-patterns - read, modify, write...",True,http://blog.2ndquadrant.com/postgresql-anti-pa...,dataradar
1,True,False,"DeepLearning,MachineLearning,NeuralNet,Library...",Engineering is the bottleneck in (Deep Learnin...,True,http://blog.dennybritz.com/2017/01/17/engineer...,dataradar
2,True,False,"Recommendation,RecommendationEngine,Recommende...","Santander Product Recommendation Competition, ...",True,http://blog.kaggle.com/2017/01/12/santander-pr...,dataradar
3,True,False,"Data,DataScience,AI",Statistics for Hackers,True,http://christopherroach.com/articles/statistic...,dataradar
4,True,False,"Python,Data,DataScience,AI",Pythonic Linear Regression,True,http://francescopochetti.com/pythonic-linear-r...,dataradar
5,True,False,"Simple,TensorFlow,DeepLearning,MachineLearning...",Simple Matrix Factorization with TensorFlow,True,http://hameddaily.blogspot.fr/2016/12/simple-m...,dataradar
6,True,False,"Test,HowTo,GettingStarted,Forecasting,TimeSeries",How To Backtest Machine Learning Models for Ti...,True,http://machinelearningmastery.com/backtest-mac...,dataradar
7,True,False,"Python,HowTo,GettingStarted,Data,DataScience",How to Use and Remove Trend Information from T...,True,http://machinelearningmastery.com/time-series-...,dataradar
8,True,True,"DeepLearning,MachineLearning,NeuralNet,Library...",State of the art deep learning model for quest...,True,http://metamind.io/research/state-of-the-art-d...,dataradar
9,True,False,"Product,Amazon,Data,DataScience,AI",Playing with 80 Million Amazon Product Review ...,True,http://minimaxir.com/2017/01/amazon-spark/,dataradar


## Uploading to FeedCrunch

In [3]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

api_route = "https://www.feedcrunch.io/api/1.0/authenticated/post/article/"
apikey_fc    = "4377190f-24a5-4ef7-a761-a877bf6218e3"
api_url = api_route + apikey_fc + "/"

restart_point = 39 # Starts at 0

for index in range(restart_point, df.shape[0]):

    fields = convert_dict_unicode2string(df.iloc[index].to_dict())
    files = {}

    data, headers = encode_multipart(fields, files)
    request = urllib2.Request(api_url, data=data, headers=headers)

    f = urllib2.urlopen(request)

    tmp_rslt = f.read()
    
    try:
        tmp_rslt = tmp_rslt.replace('true', '"true"')
        tmp_rslt = tmp_rslt.replace('false', '"false"')

        data_rslt = ast.literal_eval(tmp_rslt)
        
        
        old_stdout = sys.stdout
        log_file = open("article_posting.log","a")
        sys.stdout = log_file
        
        if data_rslt["success"] != "true":
            print "Error  : " + str(index)
        else:
            print "Success: " + str(index)
        
        sys.stdout = old_stdout
        log_file.close()
            
        time.sleep(5 * 60) # Sleep for 5 Minutes
            
    except Exception, e:
        old_stdout = sys.stdout
        log_file = open("article_posting.log","a")
        sys.stdout = log_file
        print "#############"
        print "Error at index : " + str(index)
        print "An error occured in the process: " + str(e)
        sys.stdout = old_stdout
        log_file.close()
        break